In [688]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import time

In [617]:
display_step = 300
epochs = 13
batch_size = 128
rnn_size = 128
num_layers = 3
encoding_embedding_size = 200
decoding_embedding_size = 200
learning_rate = 0.001
keep_probability = 0.5

In [618]:
lines = open('movie_lines.txt',encoding='utf-8',errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt',encoding='utf-8',errors='ignore').read().split('\n')

In [619]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    #print(_line)
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [620]:
conversation_ids = []

In [621]:
for conversation in conversations:
    _conversation= conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversation_ids.append(_conversation.split(","))

In [622]:
questions = []
answers =[]

In [623]:
for conversation in conversation_ids:
    if len(conversation)%2 == 0:
        for i in range(0,(len(conversation) - 1),2):
            questions.append(id2line[conversation[i]])
            answers.append(id2line[conversation[i+1]])

In [624]:
#cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"\'ll"," will",text)
    text = re.sub(r"\'ve"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r"[-()\"\',.?:;+=-@#<>/|]","",text)
    return text

In [625]:
#cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

#cleaning answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [626]:
threshold = 10
questionwordstoint = {}

In [627]:
word2count = {}
qnswordcount={}

In [628]:
for question in clean_questions:
    words = question.split()
    for word in words:
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [629]:
for question in clean_questions:
    words = question.split()
    for word in words:
        if word not in qnswordcount:
            qnswordcount[word] = 1
        else:
            qnswordcount[word] += 1

In [630]:
for answer in clean_answers:
    words = answer.split()
    for word in words:
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [631]:
answordcount={}

In [632]:
  for answer in clean_answers:
    words = answer.split()
    for word in words:
        if word not in answordcount:
            answordcount[word] = 1
        else:
            answordcount[word] += 1

In [633]:
qdic = {}
number =0       
for word,num in qnswordcount.items():
    if num >= threshold:
        qdic[word] = number
        number += 1

In [634]:
adic = {}
number =0 
for word,num in answordcount.items():
    if num >= threshold:
        adic[word] = number
        number += 1

In [635]:
number=0
for word,num in qnswordcount.items():
    if num >= threshold:
        questionwordstoint[number] = num
        number += 1

In [636]:
answerwordtoint = {}
number = 0
for word,num in answordcount.items():
        if num >= threshold:
                answerwordtoint[number] = num
                number += 1

In [637]:
tokens = ['<PAD>','<EOS>','<UNK>','<SOS>']

In [638]:
questionwordstoint['<PAD>']=4545
questionwordstoint['<EOS>']=4546
questionwordstoint['<UNK>']=4547
questionwordstoint['<SOS>']=4548

In [639]:
answerwordtoint['<PAD>']=4811
answerwordtoint['<EOS>']=4812
answerwordtoint['<UNK>']=4812
answerwordtoint['<SOS>']=4813

In [640]:
qdic['<PAD>']=4545
qdic['<EOS>']=4546
qdic['<UNK>']=4547
qdic['<SOS>']=4548

In [641]:
adic['<PAD>']=4811
adic['<EOS>']=4812
adic['<UNK>']=4813
adic['<SOS>']=4814

In [642]:
answerinttoword = {w_i:w for w,w_i in answerwordtoint.items()}

In [643]:
questioninttoword  = {w_i:w for w,w_i in questionwordstoint.items()}

In [644]:
questionsintoint = []
answersintoint = []

In [645]:
for question in clean_questions:
    inta = []
    for word in question.split():
        if word in qdic:
            inta.append(qdic[word])
        else:           
            inta.append(questionwordstoint['<UNK>'])
    questionsintoint.append(inta)

In [646]:
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word in adic:
            ints.append(adic[word])
        else:
            ints.append(answerwordtoint['<UNK>'])
    answersintoint.append(ints)

In [647]:
for i in range(len(answersintoint)):
        answersintoint[i].append(answerwordtoint['<EOS>'])

In [648]:
for i in range(len(questionsintoint)):
        questionsintoint[i].append(questionwordstoint['<EOS>'])

In [649]:
    maximum = len(questionsintoint[ 0 ])
    for a in questionsintoint:
        if len(a) > maximum:
            maximum = len(a)

In [650]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1,maximum+1):
        for i in enumerate(questionsintoint):
                if len(i[1]) == length:
                        sorted_clean_questions.append(questionsintoint[i[0]])
                        sorted_clean_answers.append(answersintoint[i[0]])

In [651]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length) 
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')   
    return inputs, targets, target_sequence_length, max_target_len, lr, keep_prob

In [652]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    go_id = target_vocab_to_int['<SOS>']
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    return after_concat

In [653]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,source_vocab_size,encoding_embedding_size):
    #len(adic)=source_vocab_size
    #encoding_embedding_size=200
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs,vocab_size=source_vocab_size,embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells,embed,dtype=tf.float32)
    return outputs, state

In [654]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input,target_sequence_length, max_summary_length,output_layer,
                         keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell,output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input,target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,helper,encoder_state,output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True,maximum_iterations=max_summary_length)
    return outputs

In [655]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,end_of_sequence_id,
                         max_target_sequence_length,vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell,output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,tf.fill([batch_size], start_of_sequence_id),
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,helper,encoder_state,output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True,maximum_iterations=max_target_sequence_length)
    return outputs

In [656]:
def decoding_layer(dec_input, encoder_state,target_sequence_length, max_target_sequence_length,rnn_size,num_layers, 
                   target_vocab_to_int, target_vocab_size,batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state,cells,dec_embed_input,target_sequence_length,
                                            max_target_sequence_length,output_layer,keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state,cells,dec_embeddings,target_vocab_to_int['<SOS>'],
                                            target_vocab_to_int['<EOS>'],max_target_sequence_length,target_vocab_size, 
                                            output_layer,batch_size,keep_prob)

    return (train_output, infer_output)

In [657]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,target_sequence_length,max_target_sentence_length,
      source_vocab_size, target_vocab_size,enc_embedding_size, dec_embedding_size,rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data,rnn_size,num_layers,keep_prob,source_vocab_size,enc_embedding_size)
    
    dec_input = process_decoder_input(target_data,target_vocab_to_int,batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,enc_states,target_sequence_length,max_target_sentence_length,
                                            rnn_size,num_layers,target_vocab_to_int,target_vocab_size,batch_size,keep_prob,
                                            dec_embedding_size)
    
    return train_output, infer_output

In [658]:
max_target_sentence_length = maximum

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length,lr, keep_prob = model_inputs()
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(questionwordstoint),
                                                   len(answerwordtoint),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   questionwordstoint)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [659]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [714]:
def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    #Batch targets, sources, and the lengths of their sentences together
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [713]:
"""
def get_batches(questions, answers, batch_size):
    #Batch questions and answers together
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, qdic))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, adic))
        yield pad_questions_batch, pad_answers_batch
"""

'\ndef get_batches(questions, answers, batch_size):\n    #Batch questions and answers together\n    for batch_i in range(0, len(questions)//batch_size):\n        start_i = batch_i * batch_size\n        questions_batch = questions[start_i:start_i + batch_size]\n        answers_batch = answers[start_i:start_i + batch_size]\n        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, qdic))\n        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, adic))\n        yield pad_questions_batch, pad_answers_batch\n'

In [685]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

In [717]:
train_valid_split = int(len(sorted_clean_questions)*0.15)
train_questions = sorted_clean_questions[train_valid_split:]
train_answers = sorted_clean_answers[train_valid_split:]
valid_questions = sorted_clean_questions[:train_valid_split]
valid_answers = sorted_clean_answers[:train_valid_split]
(valid_questions_batch, valid_answers_batch, valid_questions_lengths, valid_answers_lengths ) = next(get_batches(valid_questions,
                                                                                valid_answers,batch_size,qdic['<PAD>'],adic['<PAD>']))  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_questions, train_answers, batch_size,qdic['<PAD>'], adic['<PAD>'])):
                
               # _, loss = sess.run([train_op, cost],{input_data: source_batch,targets: target_batch,lr: learning_rate,
                # target_sequence_length: targets_lengths,keep_prob: 0.5})
                
                if batch_i % display_step == 0 and batch_i > 0:
                    batch_train_logits = sess.run(inference_logits,{input_data: source_batch,
                                                        target_sequence_length: targets_lengths,keep_prob: 1.0})
                    
                    #batch_valid_logits = sess.run(inference_logits,{input_data: valid_questions_batch,
                     #                                   target_sequence_length: valid_answers_lengths,keep_prob: 1.0})
                    train_acc = get_accuracy(target_batch, batch_train_logits)


In [718]:
train_acc

0.0